# Artificial Neural Network

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.9.1'

## Part 1 - Data Preprocessing

### Importing the dataset

In [3]:
%cd Z:\PPMI_Data\Excels\CollaborativeFiltering

Z:\PPMI_Data\Excels\CollaborativeFiltering


,APPRDX,Anxiety,Apathy,Benton,Clock,Cognition,COGSTATE,Constipate,Depress,DopaDefic,...,Semantic,SleepDay,SleepNight,STAIA,STAIS,Symbol_Digit,Trail_Making_A,Trail_Making_B,UPSIT,Urine
0,1,1,0,12.16,7,1,1,1,1,0,...,57,1,0,45,59,47.5,110,204,17,1
1,1,1,1,7.76,6,1,1,0,0,0,...,36,1,3,40,39,52.0,27,52,9,0


## Save model

In [20]:
# Save model
ann.save("model_ann.h5")

In [21]:
ann.history

## Load an empty dataset

In [54]:
df2 = pd.read_csv('NonMotor_Empty.csv')
# Removing a column
df3 = df2.drop('Patient_ID', axis = 1)
df3.head(2)

,Anxiety,Apathy,Benton,Clock,Cognition,COGSTATE,Constipate,Depress,DopaDefic,Education,...,Semantic,SleepDay,SleepNight,STAIA,STAIS,Symbol_Digit,Trail_Making_A,Trail_Making_B,UPSIT,Urine
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Take input from user

In [60]:
def update_values(df3):
    # Input from the user for number of columns
    num_columns = int(input("Enter the number of columns you want to update (minimum 3, maximum 45): "))
    
    while num_columns < 3 or num_columns > 45:
        print("Invalid number of columns. Please enter a number between 3 and 45.")
        num_columns = int(input("Enter the number of columns you want to update (minimum 3, maximum 45): "))
    
    columns = []
    values = []
     
     # Loop to get input from the user for each column
    for i in range(num_columns):
        column = input("Enter the name of the Feature %d: " % (i + 1))
        value = input("Enter the value to update for Feature %d: " % (i + 1))
        
        columns.append(column)
        values.append(int(value))
    
    # Update the values in the dataframe
    for i in range(num_columns):
        df3.loc[df3[columns[i]] != values[i], columns[i]] = values[i]
    
    return df3

# Call the function
df3 = update_values(df3)

# Print the updated dataframe
df3.head()

Enter the number of columns you want to update (minimum 3, maximum 45): 4
Enter the name of the Feature 1: UPSIT
Enter the value to update for Feature 1: 3
Enter the name of the Feature 2: COGSTATE
Enter the value to update for Feature 2: 3
Enter the name of the Feature 3: Semantic
Enter the value to update for Feature 3: 6
Enter the name of the Feature 4: Apathy
Enter the value to update for Feature 4: 2


,Anxiety,Apathy,Benton,Clock,Cognition,COGSTATE,Constipate,Depress,DopaDefic,Education,...,Semantic,SleepDay,SleepNight,STAIA,STAIS,Symbol_Digit,Trail_Making_A,Trail_Making_B,UPSIT,Urine
0,NaN,2.0,NaN,6.0,NaN,3.0,NaN,3.0,NaN,NaN,...,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN


In [59]:
# Identify the columns to delete
value = 'NaN'
to_drop = df3.columns[df3.isin([value]).any()].tolist()
df4 = df3.drop(to_drop, axis=1)
df4.head()

,Anxiety,Apathy,Benton,Clock,Cognition,COGSTATE,Constipate,Depress,DopaDefic,Education,...,Semantic,SleepDay,SleepNight,STAIA,STAIS,Symbol_Digit,Trail_Making_A,Trail_Making_B,UPSIT,Urine
0,NaN,3.0,NaN,6.0,NaN,NaN,NaN,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Scale Transformation

df3 = sc.transform(df3)

df3

### Remove columns empty values with NaN

# Perform Transfer Learning

In [61]:
# Create a custom layer to wrap the pre-trained model

def build(self, batch_input_shape):
    self.model.build(batch_input_shape)
    super(WrapperLayer, self).build(batch_input_shape)
    
def call(self, inputs):
    return self.model(inputs)

input_layer = tf.keras.Input(shape=df3.shape[1:])

# Load a pre-trained model
model = load_model("model_ann.h5")

# Wrap the pre-trained model in the custom layer
x = WrapperLayer(model)(input_layer)

# Create a new model using the custom input layer and the output from the wrapped pre-trained model
model = tf.keras.models.Model(inputs=input_layer, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [75]:
prediction = model.predict(df3)
if prediction[0][0] > 0.5:
    predicted_class = "Patient"
else:
    predicted_class = "Healthy"

# Output the result to the user
print("The predicted class is:", predicted_class)

The predicted class is: Healthy


    The class definition starts with the class name "WrapperLayer" which inherits from the "Layer" class in the "tf.keras.layers" module.
    The "init" function takes a pre-trained model as input and calls the parent class's "init" function using "super(WrapperLayer, self).init()".
    The "call" function takes inputs and returns the output of the pre-trained model when called. This allows the WrapperLayer to behave as if it were the pre-trained model, when it is used as part of a larger model.

The 0.5 in the line x = tf.keras.layers.Dropout(0.5)(x) is the rate of dropout, meaning that on average, during training, half of the neurons will be randomly dropped out (i.e., ignored), forcing the network to be more robust and prevent overfitting. Dropout is a common regularization technique for deep learning models. The value 0.5 is a common default for the dropout rate, but this value can be tuned based on the specifics of the problem and the data.

The number 1024 in the dense layer represents the number of neurons/units in the layer. It is a hyperparameter that can be tuned to optimize the performance of the model. The activation function used in this layer is 'relu' (rectified linear unit) which is a popular activation function used in deep learning models. The purpose of using this activation function is to introduce non-linearity in the model, allowing it to learn more complex representations of the data.b

The size of the dense layer, in this case 1024, is a hyperparameter that can be tuned. The number of neurons in the dense layer can have an impact on model performance, with too few neurons leading to underfitting and too many neurons leading to overfitting.

The number of neurons in the dense layer can be increased when the model needs to capture more complex relationships between the input and output. This can happen when the input data is high dimensional, has a large number of features, or when the relationship between the input and output is non-linear. Increasing the number of neurons in the dense layer can also improve model performance when the model has low capacity, meaning that it is not able to fit the data well with a small number of neurons.

On the other hand, the number of neurons in the dense layer can be decreased when the model is overfitting, meaning that it is fitting the training data too well and is not generalizing well to new data. Decreasing the number of neurons in the dense layer can simplify the model and help it to generalize better to new data.

Ultimately, the optimal number of neurons in the dense layer depends on the specific task and data, and finding the right number can require some trial and error. It is common to use techniques such as cross-validation and grid search to find the best hyperparameters for a particular task.

x = tf.keras.layers.Flatten()(x) is a line of code that uses the Flatten layer from TensorFlow's tf.keras API. The purpose of this layer is to flatten the input tensor into a 1D array.

Before flattening, the input tensor might have multiple dimensions (such as height, width, and depth dimensions in a convolutional neural network). The Flatten layer takes in this tensor and reshapes it into a 1D array, which is then used as input to the next layer in the model.